In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
dataset_path = '../datasets/validation/'

RD_CRS = 'epsg:28992'
LL_CRS = 'WGS84'

In [ ]:
area = gpd.read_file(f'{dataset_path}correct_trees_areas_oosterpark.shp', crs=RD_CRS)
trees = gpd.read_file(f'{dataset_path}correct_trees_oosterpark.shp', crs=LL_CRS)

In [ ]:
trees.to_crs(RD_CRS, inplace=True)

In [ ]:
%matplotlib widget

fig, ax = plt.subplots(1, figsize=(8,5))

area.plot(ax=ax, color='grey', alpha=0.5)
trees.plot(ax=ax, color='darkgreen', marker='.', markersize=5, alpha=1)